In [1]:
import os
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv("synthetic_expiry_labels.csv")
label_map = {"expired": 0, "expiring_soon": 1, "fresh": 2}
df["label"] = df["freshness_class"].map(label_map)


In [3]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42, stratify=df["label"])


In [4]:
class GroceryImageDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df.loc[idx, "filename"])
        image = Image.open(img_path).convert("RGB")
        label = self.df.loc[idx, "label"]

        if self.transform:
            image = self.transform(image)

        return image, label


In [5]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load data and encode classes
df = pd.read_csv("synthetic_expiry_labels.csv")
label_map = {"expired": 0, "expiring_soon": 1, "fresh": 2}
df["label"] = df["freshness_class"].map(label_map)

# Train/test split
train_df, test_df = train_test_split(df, test_size=0.2, stratify=df["label"], random_state=42)


In [6]:
import os
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms

class GroceryDataset(Dataset):
    def __init__(self, dataframe, img_dir, transform=None):
        self.df = dataframe.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.df.loc[idx, "filename"])
        label = self.df.loc[idx, "label"]
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)
        return image, label

# Transforms
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.2),
    transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
    transforms.ToTensor(),
    transforms.Normalize([0.5]*3, [0.5]*3)
])

# Dataloaders
train_data = GroceryDataset(train_df, "images", transform)
test_data = GroceryDataset(test_df, "images", transform)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32)


In [7]:
import torch.nn as nn
import torch.nn.functional as F

class ExpiryCNN(nn.Module):
    def __init__(self):
        super(ExpiryCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.dropout = nn.Dropout(0.4)
        self.fc1 = nn.Linear(128 * 16 * 16, 256)
        self.fc2 = nn.Linear(256, 3)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16)
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x




In [8]:
import torch
import torch.optim as optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = ExpiryCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [10]:
import torch.nn as nn

# Class counts from your previous confusion matrix
# Class 0: 175, Class 1: 198, Class 2: 559

class_weights = torch.tensor([3.2, 2.8, 1.0]).to(device)  # Weight less frequent classes more

criterion = nn.CrossEntropyLoss(weight=class_weights)


In [11]:
# Re-initialize model
model = ExpiryCNN().to(device)

# Define criterion with class weights (as above)
# Redefine optimizer as usual
# Then start training loop again


In [12]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.0005)


In [14]:
# Basic dataset info
print("Dataset shape:", df.shape)
print("\nDataset Info:")
df.info()

# Check for missing values
print("\nMissing values:\n", df.isnull().sum())

# Summary statistics
df.describe()


Dataset shape: (4657, 4)

Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4657 entries, 0 to 4656
Data columns (total 4 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   filename         4657 non-null   object
 1   expiry_date      4657 non-null   object
 2   freshness_class  4657 non-null   object
 3   label            4657 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 145.7+ KB

Missing values:
 filename           0
expiry_date        0
freshness_class    0
label              0
dtype: int64


,label
count,4657.000000
mean,1.411209
std,0.786420
min,0.000000
25%,1.000000
50%,2.000000
75%,2.000000
max,2.000000


In [20]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (preds == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 59.87%


In [24]:
from PIL import Image
from torchvision import transforms

# Image preprocessing
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

# Load and preprocess image
img = Image.open('test.png').convert('RGB')
img_tensor = transform(img).unsqueeze(0).to(device)  # Add batch dimension

# Predict
model.eval()
with torch.no_grad():
    output = model(img_tensor)
    _, predicted_class = torch.max(output, 1)

print(f"Predicted Class: {predicted_class.item()}")


Predicted Class: 2
